In [2]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

### Load IMDB data set

In [3]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84131840/84125825 [==============================] - 339s 4us/step


['imdbEr.txt', 'README', 'imdb.vocab', 'test', 'train']

In [4]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'unsupBow.feat',
 'urls_pos.txt',
 'pos',
 'urls_unsup.txt',
 'neg',
 'urls_neg.txt',
 'unsup']

In [5]:

remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Use the train directory to create both train and validation datasets with a split of 20% for validation.

In [6]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)


In [15]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.00788599,  0.00081977,  0.00781255, -0.04228721, -0.04783029],
       [-0.03026186,  0.03411737,  0.03773388, -0.03534966,  0.03944583],
       [-0.01677812, -0.02430487,  0.04912808, -0.01094632, -0.04946957]],
      dtype=float32)

In [16]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

In [18]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                    '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)



In [19]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [20]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [22]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 3s 133ms/step - loss: 0.6927 - accuracy: 0.5002 - val_loss: 0.6902 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 92ms/step - loss: 0.6886 - accuracy: 0.5002 - val_loss: 0.6842 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 89ms/step - loss: 0.6813 - accuracy: 0.5002 - val_loss: 0.6746 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 88ms/step - loss: 0.6697 - accuracy: 0.5002 - val_loss: 0.6604 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 91ms/step - loss: 0.6529 - accuracy: 0.5002 - val_loss: 0.6413 - val_accuracy: 0.4886
Epoch 6/15
20/20 [==============================] - 2s 89ms/step - loss: 0.6307 - accuracy: 0.5002 - val_loss: 0.6180 - val_accuracy: 0.4906
Epoch 7/15
20/20 [==============================] - 2s 89ms/step - loss: 0.6037 - accuracy: 0.5204 - val_loss: 0.5916 - val_accuracy: 0.5740
Epoch 8/15
2

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_1 (TextVe (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [25]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [28]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue  # skip 0, it's padding.
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()